## Code used to georefence and evaluate the results 

In [ ]:
# import libraries
import numpy as np
import tensorflow as  tf
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

from datetime import datetime
from packaging import version

%tensorflow_version 2.x
from tensorflow import keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."

In [ ]:
# Install segmentation_models
pip install segmentation_models

In [ ]:
import segmentation_models as sm

In [ ]:
# Define TP metric
def true_positive(y_true, y_pred):
    y_pred_pos = tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    tp = tf.keras.backend.sum(y_pos * y_pred_pos)


    return tp * (0.005**2)

In [ ]:
# Define TN metrics
def true_negative(y_true, y_pred):
    y_pred_pos = tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    
    tn = tf.keras.backend.sum(y_neg * y_pred_neg)


    return tn * (0.005**2)

In [ ]:
# Define FP metrics
def false_positve(y_true, y_pred):
    y_pred_pos = tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

   
    fp = tf.keras.backend.sum(y_neg * y_pred_pos)
   

    return fp * (0.005**2)

In [ ]:
# Define FN metrics
def false_negative(y_true, y_pred):
    y_pred_pos = tf.keras.backend.round(tf.keras.backend.clip(y_pred, 0, 1))
    y_pred_neg = 1 - y_pred_pos

    y_pos = tf.keras.backend.round(tf.keras.backend.clip(y_true, 0, 1))
    y_neg = 1 - y_pos

    fn = tf.keras.backend.sum(y_pos * y_pred_neg)



    return fn * (0.005**2)

In [ ]:
# Network Metrics
metrics = [sm.metrics.Precision(threshold=0.5),sm.metrics.Recall(threshold=0.5),sm.metrics.FScore(threshold=0.5,beta=1),sm.metrics.IOUScore(threshold=0.5),matthews_correlation,true_positive,true_negative,false_positve,false_negative]

In [ ]:
# Define model Architecture
def Unet(lr,filtersFirstLayer, pretrained_weights = None,input_size = (32,32,6)):
    inputs = Input(input_size)
    
    conv1 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same')(inputs)
    conv1 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
   
    conv2 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same')(pool1)
    conv2 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)


    conv3 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same')(pool2)
    conv3 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

 
    conv4 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same')(pool3)
    conv4 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

  
    conv5 = Conv2D(filtersFirstLayer*16, 3, activation = 'relu', padding = 'same')(pool4)
    conv5 = Conv2D(filtersFirstLayer*16, 3, activation = 'relu', padding = 'same')(conv5)

    # Up 1 
    up6 = Conv2D(filtersFirstLayer*8, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv5))
    merge6 = concatenate([conv4,up6], axis = 3)
    conv6 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same')(merge6)
    conv6 = Conv2D(filtersFirstLayer*8, 3, activation = 'relu', padding = 'same')(conv6)

    # Up 2 
    up7 = Conv2D(filtersFirstLayer*4, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same')(merge7)
    conv7 = Conv2D(filtersFirstLayer*4, 3, activation = 'relu', padding = 'same')(conv7)

    # Up 3 
    up8 = Conv2D(filtersFirstLayer*2, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same')(merge8)
    conv8 = Conv2D(filtersFirstLayer*2, 3, activation = 'relu', padding = 'same')(conv8)

    # Up 4 
    up9 = Conv2D(filtersFirstLayer, 2, activation = 'relu', padding = 'same')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same')(merge9)
    conv9 = Conv2D(filtersFirstLayer, 3, activation = 'relu', padding = 'same')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same')(conv9)

    # output
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs, conv10)

    model.compile(optimizer = Adam(lr = lr), loss = 'binary_crossentropy', metrics = metrics)
    
    model.summary()

    if(pretrained_weights):
    	model.load_weights(pretrained_weights)

    return model

model = Unet(0.0001,32,input_size=(1024,1024,6))

In [ ]:
# Load model weights
model.load_weights("/content/drive/My Drive/Mestrado/artigo/artigo_final/results/augmented_terrain/64_64/random/model/unet/unet_random_size_64_filters_32_batch_size_16_lr_0.001.hdf5")

In [ ]:
# Evaluate the model 
model.evaluate(X_test,Y_test)

In [ ]:
# Make the prediction over the test areas and save as .tif

# import rasterio
import rasterio


# Predict
preds_train = model.predict(X_test, verbose=1)
# Assign the value one for predictions with results > 0.5
preds_train_t = (preds_train > 0.5).astype(np.uint8)
# Open the original image
dataset = rasterio.open(original_image_path)
# Get the metadata from the image
meta = dataset.meta
# Change the metadata to 1 (Result image will have just one channel)
meta["count"] = 1
# Nodata values = 0
meta["nodata"] = 0
# data type = uint8 (8bits)
meta["dtype"] = "uint8"
# Ajust array dimensions
save = np.squeeze(preds_train_t, axis=(0,3))
save = np.expand_dims(save,axis = 0)
# Save .tif image
with rasterio.open("area_2_32_predict.tif", 'w', **meta) as dst:
    dst.write(save)